In [3]:
import pandas as pd
import numpy as np

data1 = pd.read_csv('../database/pseudo_observations/1321.txt')
data1 = np.array(data1.iloc[::2, :])
data1 = np.array([float(i[0]) for i in data1])

data2 = pd.read_csv('../database/pseudo_observations/AUD.txt')
data2 = np.array(data2.iloc[::2, :])
data2 = np.array([float(i[0]) for i in data2])

data3 = pd.read_csv('../database/pseudo_observations/EUR.txt')
data3 = np.array(data3.iloc[::2, :])
data3 = np.array([float(i[0]) for i in data3])

data4 = pd.read_csv('../database/pseudo_observations/GLD.txt')
data4 = np.array(data4.iloc[::2, :])
data4 = np.array([float(i[0]) for i in data4])

data5 = pd.read_csv('../database/pseudo_observations/HYG.txt')
data5 = np.array(data5.iloc[::2, :])
data5 = np.array([float(i[0]) for i in data5])

data6 = pd.read_csv('../database/pseudo_observations/IEAC.txt')
data6 = np.array(data6.iloc[::2, :])
data6 = np.array([float(i[0]) for i in data6])

data7 = pd.read_csv('../database/pseudo_observations/IVV.txt')
data7 = np.array(data7.iloc[::2, :])
data7 = np.array([float(i[0]) for i in data7])

data8 = pd.read_csv('../database/pseudo_observations/JPY.txt')
data8 = np.array(data8.iloc[::2, :])
data8 = np.array([float(i[0]) for i in data8])

data9 = pd.read_csv('../database/pseudo_observations/LQD.txt')
data9 = np.array(data9.iloc[::2, :])
data9 = np.array([float(i[0]) for i in data9])

data10 = pd.read_csv('../database/pseudo_observations/MSE.txt')
data10 = np.array(data10.iloc[::2, :])
data10 = np.array([float(i[0]) for i in data10])

data11 = pd.read_csv('../database/pseudo_observations/vas.txt')
data11 = np.array(data11.iloc[::2, :])
data11 = np.array([float(i[0]) for i in data11])

In [34]:
import pyvinecopulib as pv
controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian,
                                             pv.BicopFamily.clayton,
                                             pv.BicopFamily.frank,
                                             pv.BicopFamily.gumbel,
                                             pv.BicopFamily.student])

u = np.stack([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10, data11]).T
#cop = pv.Vinecop(u, controls=controls)

#u_sim = cop.simulate(100)

#np.savetxt("u_sim.txt", u_sim, delimiter=",")
#print(u_sim)

[[0.05806084 0.10830092 0.79261315 ... 0.28698886 0.21304408 0.06698927]
 [0.19199837 0.62810547 0.4585476  ... 0.49344329 0.57616201 0.60300312]
 [0.85327287 0.52715455 0.69477566 ... 0.63470831 0.69612464 0.29218895]
 ...
 [0.12185408 0.55500805 0.97048421 ... 0.10141121 0.0330822  0.05961305]
 [0.1097628  0.92412216 0.49309428 ... 0.63394547 0.30222343 0.50744772]
 [0.82957789 0.0398039  0.71617357 ... 0.63057063 0.8447574  0.66376797]]


In [42]:
print(cop.str())

** Tree: 0
2,8 <-> Student, parameters = 0.287528
       2
3,4 <-> Student, parameters = 0.770984
       2
4,8 <-> Student, parameters = -0.854454
  9.36866
6,9 <-> Student, parameters = 0.972206
  5.0819
8,1 <-> Student, parameters = 0.787825
 37.9082
9,5 <-> Student, parameters = 0.956871
 4.99121
1,10 <-> Student, parameters = 0.62582
      2
5,7 <-> Student, parameters = 0.905742
 10.7949
10,7 <-> Student, parameters = 0.869476
 5.03054
7,11 <-> Student, parameters = 0.850142
  5.0797
** Tree: 1
2,1 | 8 <-> Frank, parameters = -0.136243
3,8 | 4 <-> Gumbel 270°, parameters = 1.80523
4,1 | 8 <-> Gumbel 180°, parameters = 1.053
6,5 | 9 <-> Gumbel, parameters = 1.52942
8,10 | 1 <-> Gumbel, parameters = 1.31604
9,7 | 5 <-> Frank, parameters = -2.65685
1,7 | 10 <-> Gumbel, parameters = 1.13241
5,11 | 7 <-> Gumbel, parameters = 1.41704
10,11 | 7 <-> Gumbel 180°, parameters = 1.13977
** Tree: 2
2,4 | 1,8 <-> Gaussian, parameters = 0.0381867
3,1 | 8,4 <-> Frank, parameters = 0.93519
4,10 | 

In [19]:
from database.database_parser import create_connection
from database.database_parser import select_curr_solution

test_connection = create_connection('../database/abacus_database.db')

In [20]:
from database.database_parser import select_price_data

def generate_volatility(params, data):
    n_observations = len(data)
    initial_squared_vol_estimate = (params[0]
                                        + params[1] * (data[0] ** 2)
                                        + params[3] * (data[0] ** 2) * np.where(data[0] < 0, 1, 0)
                                        + params[2] * (data[0] ** 2))
    current_squared_vol_estimate = initial_squared_vol_estimate

    for i in range(0, n_observations):
        current_squared_vol_estimate = (params[0] + params[1] * (data[i - 1] ** 2)
                                            + params[3] * (data[i - 1] ** 2) * np.where(data[i - 1] < 0, 1, 0)
                                            + params[2] * current_squared_vol_estimate)

    return np.sqrt(current_squared_vol_estimate)

def get_volatility(asset: str):
    opt_sol = select_curr_solution(test_connection, asset)
    adj_close = select_price_data(test_connection, '1321')
    log_returns = np.array(np.log(adj_close / adj_close.shift(1))[1:])
    return generate_volatility(params=opt_sol, data=log_returns)

In [7]:
from distributions.student_poisson_mixture import spm
# Calculate all inverse samples.
def generate_samples(us, asset):
    opt_sol = select_curr_solution(test_connection, asset)
    mu = opt_sol[4]
    sigma = get_volatility(asset)[0]
    kappa = opt_sol[5]
    lamb = opt_sol[6]
    nu = opt_sol[7]

    for p in us:
        s = spm.ppf(p, mu, sigma, kappa, lamb, nu)
        print(s.root)

generate_samples(u_sim[:, 4], 'IVV')

1.0584664836470876
0.29376358048548606
-0.018547013236331327
-0.018540169082710785
0.9373604137841106
-0.009370369113802086
-0.11197432799522992
-0.014153200520035035
0.01968628690954487
0.048521438908058705
-0.034856268800176095
-0.018702032196939347
0.009192375284793185
-0.04457269369490421
0.5549821022052526
-0.030350200842128368
0.3424838732209676
0.04803011991686046
-0.07242956460401717
0.029592015512437717
0.061801177424709794
0.12858739572443773
-0.04321951457804534
-0.042140300131046894
0.07754664967959247
-0.044797028327946076
0.09282195310411946
-0.019354058068346197
0.07639690173581754
-0.06956005699463562
-0.019735101098384895
0.04194976724074194
0.09362345430676913
-0.04692863344541148
-0.025877356333441622
0.07485770663120202
-0.5855520035975932
0.8069473200891641
0.013652933819617517
-0.04390634923081404
0.014466340724592618
-0.009486637105158043
-0.006765109687165978
-0.013501740080299217
0.18576803323188584
-0.04282738737880735
0.05019180971988053
0.06142885116119932
0

ValueError: f(a) and f(b) must have different signs

In [10]:
df_ret = pd.read_csv('../database/return_samples/IVV.txt')
len(df_reta)

76